In [2]:
!pip install BeautifulSoup4

     |████████████████████████████████| 112kB 17.3MB/s eta 0:00:01


In [3]:
!pip install requests

In [4]:
!pip install lxml

     |████████████████████████████████| 5.8MB 18.3MB/s eta 0:00:01


In [5]:
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html.parser')

#print(table)

table = soup.table
header = table.find_all('th')
#column header list = headList
header_list = [i.text for i in header]
#print(header_list)

data_list = []
row =[]
for i in table.select('tr')[1:]:
    for j in i.select('td'):
        row.append(j.text)
    data_list.append(row)
    row=[]

df = pd.DataFrame(data = data_list, columns=header_list)
df.head()
df.shape

(180, 3)

In [6]:
list = df.values.tolist()
list[0:5]

[['M1A\n', 'Not assigned\n', '\n'],
 ['M2A\n', 'Not assigned\n', '\n'],
 ['M3A\n', 'North York\n', 'Parkwoods\n'],
 ['M4A\n', 'North York\n', 'Victoria Village\n'],
 ['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n']]

In [7]:
list_new = np.array(list)
convert_list = np.char.strip(list_new , "\n")

In [8]:
data = pd.DataFrame(convert_list, columns={'Postal Code', 'Borough','Neighborhood'})
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
data.replace('', np.nan, inplace = True)
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Data Wrangling

In [10]:
d = data.values.tolist()
dN= []
i = 0 
N = len(data['Borough'])
for i in range(N):
    if d[i][1] == 'Not assigned':
        i = i +1 
    elif d[i][1] != 'Not assigned' and d[i][2] == 'NaN':
        dN.append([d[i][0],d[i][1],d[i][1]])
        i = i + 1
    else: 
        dN.append(d[i])
        i = i +1 

In [11]:
#convert into DataFrame 
# DataN is the Dataframe name of the clean file which satisfy the following condition:
# *Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#*If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
DataN = pd.DataFrame(dN, columns ={'Postal Code', 'Borough', 'Neighborhood'})
data1 = DataN.sort_values(by=['Postal Code'], ascending = True)
data1.head()

,Postal Code,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [12]:
#importing the geospatial coordinate in csv
filename = 'Geospatial_Coordinates (2).csv'
geo = pd.read_csv(filename)
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#transform the dataframe DataN into an array dfL
dfL = data1.values.tolist()
#transfrom the dataframe geo into an array gfL
gfL = geo.values.tolist()
dfL[0][0]

'M1B'

In [14]:
postal = data1['Postal Code'].values.tolist()
data_final = []
i = 0
N = len(postal)
for i in range(N):
    if gfL[i][0] == postal[i]:
        data_final.append([postal[i], dfL[i][1], dfL[i][2], gfL[i][1], gfL[i][2]])
        i = i + 1 
    else:
        i = i + 1
        
data_final = pd.DataFrame(data_final)
data_final = data_final.rename(columns={0:'Postal Code',1:'Borough',2:'Neighborhood',3:'Latitude',4:'Longitude'})
data_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
